<a href="https://colab.research.google.com/github/y191179/arduino-test/blob/main/datamodels_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports & functions
import torch as ch
from typing import List
import numpy as np
import torchvision
from torch.utils.data import Subset
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import gdown

def load_datamodels(file_path: str):
    # Load the saved file
    data = ch.load(file_path,  weights_only=True)

    # Access the contents
    weight = data['weight']
    bias = data['bias']
    lam = data['lam']

    return weight, bias, lam


def get_binary_labels(dataset, animate_labels: set = {2, 3, 4, 5, 6, 7}) -> List[bool]:
    binary_targets = [label in animate_labels for label in dataset.targets]
    return binary_targets

**Experiment**

In this experiment, we trained circa 60,000 classifiers on random subsets of the CIFAR10 training set (alpha = 0.1) on the task to predict animate vs inanimate objects. This is a comparatively simple task compared to learning to predict all 10 classes (dogs, airplanes, e.t.c ...) and allows us to investigate model class behavior with some information on known sub-classes in the dataset for which models will likely have learnt different patterns (when trained on random subsets of the data) to solve this task.

**Why are we using this 'toy' dataset and not biomedical data?**

In using this framework to uncover underlying subclasses from biomedical datasets, we will likely not have access to such ground-truth labels. Because we do have these labels for CIFAR10 and because the data are easy to visualize, we can quickly start to explore what features the models may be using to solve a binary classification task with datamodels in this example.

**Note**

Note that to facilitate speed in training these models, we only used 50% of the CIFAR10 training set from which to sample subsets of training datapoints (since this is a relatively simple task) and our datamodels are computed 'only' on 60,000 models (compare to results from Ilyas et al on the full dataset 10-class classifiers trained on >1 M models: https://arxiv.org/pdf/2202.00622).

In [ ]:
# get datamodels
file_id = '1YJDHCAI56AXJ6fv3PSRucyFkskcWPxqt'
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, 'datamodels.pt', quiet=False)

# get datamodels outputs
datamodels =load_datamodels('datamodels.pt')
weights = datamodels[0].cpu().numpy()
bias = datamodels[1].cpu().numpy()

# load the CIFAR10 dataset
ds_test = torchvision.datasets.CIFAR10('.', train=False, download=True)
ds_train = torchvision.datasets.CIFAR10('.', train=True, download=True)


Downloading...
From (original): https://drive.google.com/uc?id=1YJDHCAI56AXJ6fv3PSRucyFkskcWPxqt
From (redirected): https://drive.google.com/uc?id=1YJDHCAI56AXJ6fv3PSRucyFkskcWPxqt&confirm=t&uuid=dcc668ce-e19b-43fc-9696-294aaab89f5d
To: /content/datamodels.pt
100%|██████████| 1.00G/1.00G [00:09<00:00, 104MB/s]


100%|██████████| 170498071/170498071 [00:03<00:00, 47451447.53it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


**First, we'll have a look at a lower dimensional projection of the datamodels embedding.**

The datamodels embedding of a given input (here the test CIFAR10 dataset on which we evaluated model performance trained on different subsets of the train dataset) into $\mathbb{R}^d$, where $d$ is the size of the training set (here 25,000). We'll compare the cumulative variance explained by the first N prinicpal components for projections of datamodel embeddings to projections of an equal number of randomly sampled pixels from the original dataset.

- *What do you notice?*

In [ ]:
Npcs = 500

cifar_data = np.array(ds_test.data)
# Pick the same random pixels for each sample
random_pixel_indices = np.random.choice(cifar_data.shape[1] * cifar_data.shape[2] * cifar_data.shape[3], 1000, replace=False)
# Reshape the data to be a 2D array where each row is a sample and each column is a pixel
cifar_data_reshaped = cifar_data.reshape(cifar_data.shape[0], -1)
# Select the random pixels for each sample
selected_pixels = cifar_data_reshaped[:, random_pixel_indices] # 1000 random pixels for 10000 samples in the test set
# compute projections
pca = PCA(n_components=Npcs)
pca_result = pca.fit_transform(selected_pixels)
# Plot the cumulative explained variance ratio of the principal components
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(4, 3))
plt.plot(range(0, Npcs), cumulative_variance, marker='o', linestyle='--', alpha=0.7, markersize=1, label='projection from pixel')


# Perform PCA on the weights
pca = PCA(n_components=Npcs)
temp = weights.T[:,:1000] # 1000 datamodel embeddings for 10000 samples in the test set
x = (temp - np.mean(temp, axis=0)) / np.std(temp, axis=0)
pca_result = pca.fit_transform(temp)
# Plot the cumulative explained variance ratio of the principal components
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(0, Npcs), cumulative_variance, marker='o', linestyle='--', alpha=0.7, markersize=1, label='projection from datamodels embeddings')
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()
plt.show()

The datamodel embeddings seem to spread out the variance over a high number of dimensions,leading to a high effective dimensionality.

**Now we'll perform PCA on the full datamodels embedding and have a look at the principal components.**

This should start to give you an intution for what features the models may be learning to solve the classification task, for different subsets of the population.

- *What do you notice when you look at the first few PCs computed on all the test samples?*

- *What do you notice when we look at the first few PCs computed on only one of the classes we trained on (inanimate vs animate)?*

In [ ]:
pca = PCA(n_components=20)
temp = weights.T
x = (temp - np.mean(temp, axis=0)) / np.std(temp, axis=0)
pca_result_all = pca.fit_transform(x)


In [ ]:
# project the full test data
PC = 0 # explore different PCs

indices_sorted = np.argsort(pca_result_all[:,PC])

fig, axes = plt.subplots(3, 10, figsize=(10, 3))
for i, ax in enumerate(axes.flat):
    ax.imshow(ds_test.data[indices_sorted][i])
    ax.axis('off')
fig.suptitle(f'Negative extreme of PC {PC}', fontsize=16)
plt.show()

fig, axes = plt.subplots(3, 10, figsize=(10, 3))
for i, ax in enumerate(axes.flat):
    ax.imshow(ds_test.data[indices_sorted][-(i+1)])
    ax.axis('off')
fig.suptitle(f'Positive extreme of PC {PC}', fontsize=16)
plt.show()



In [ ]:
get_animate=False #### Set True or False for the test class you want to compute PC projections for ####

if get_animate:
  pca = PCA(n_components=20)
  indices = np.array(get_binary_labels(ds_test))
  temp = weights.T[indices]
  x = (temp - np.mean(temp, axis=0)) / np.std(temp, axis=0)
  pca_result = pca.fit_transform(x)
  subset = ds_test.data[indices]
else:
  pca = PCA(n_components=20)
  indices = np.invert(get_binary_labels(ds_test))
  temp = weights.T[indices]
  x = (temp - np.mean(temp, axis=0)) / np.std(temp, axis=0)
  pca_result = pca.fit_transform(x)
  subset = ds_test.data[indices]


In [ ]:
PC = 2 #### Have a look at a few PCs ####

indices_sorted = np.argsort(pca_result[:,PC])

fig, axes = plt.subplots(3, 10, figsize=(10, 3))
for i, ax in enumerate(axes.flat):
    ax.imshow(subset[indices_sorted][i])
    ax.axis('off')
fig.suptitle(f'Negative extreme of PC {PC}', fontsize=16)
plt.show()

fig, axes = plt.subplots(3, 10, figsize=(10, 3))
for i, ax in enumerate(axes.flat):
    ax.imshow(subset[indices_sorted][-(i+1)])
    ax.axis('off')
fig.suptitle(f'Positive extreme of PC {PC}', fontsize=16)
plt.show()



**Now let's have a look at the top influencing training images for a given test set image**

(i.e. the training images with the largest positive weights in the datamodels matrix for a given test image).

- *What about the top negative influencers (i.e. images that make the model confident in an incorrect prediction)? What do you expect these images to look like?*

- *What about train images that don't seem to influence the models predictions?*

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=10, figsize=(20, 4))  # 2 rows, 10 columns, adjust figure size

for test_id in range(10):
    # Test Image in the first row (first row, test_id-th column)
    axes[0, test_id].imshow(ds_test.data[test_id])  # Plot test image
    axes[0, test_id].set_title('Test Image')  # Set title for the test image
    axes[0, test_id].axis('off')  # Turn off axis

    # Top Influencer in the second row (second row, test_id-th column)
    influencer_id = np.argmax(weights[:, test_id])  # Get the index of the top influencer
    axes[1, test_id].imshow(ds_train.data[influencer_id])  # Plot top influencer image
    axes[1, test_id].set_title('Top Influencer')  # Set title for the top influencer
    axes[1, test_id].axis('off')  # Turn off axis

# Adjust layout to prevent overlap and improve spacing
plt.tight_layout()
plt.show()

- *What happens if we try to cluster on the datamodels embeddings? (We'll just use the top PCs to make this faster).
What do you notice?*


In [ ]:
# Perform KMeans Clustering on the PCA result
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans_result = kmeans.fit_predict(pca_result_all)

for i in range(4):
    fig, axes = plt.subplots(6, 15, figsize=(15, 5))
    indices = np.argwhere(kmeans_result==i).reshape(-1)
    for x, ax in enumerate(axes.flat):
        ax.imshow(ds_test.data[indices[x]])
        ax.axis('off')
    fig.suptitle(f'Kmeans cluster {i}', fontsize=16)
    plt.show()

**Finally, we'll run a simple experiment to understand how the datamodels embeddings may be useful to new tasks.**

Instead of using the images as features to train some classifier, we can now use our datamodel weights (i.e. the influencers) as embeddings (i.e. a representation of our data), which - as we've seen - are much more interpretable than the pixels we started from.

As an illustration of this, let's do the following:

- In the first instance, we'll train a linear classifier on a random sample of pixels from the CIFAR10 test data as features. We'll split these data into train and validation sets and fit a linear classifier to predict a single subclass from these data (see below for available subclasses).

- In the second instance, we'll train a linear classifier on a subset of datamodel embeddings. We'll again split these data into train and validation sets and fit a linear classifier to predict a single subclass from these data.

Try varying the numbers of features, different target subclasses, and training size.

- *What do you notice as you increase the number of features?*

- *What do you notice when you use all available features? And what if you also dramatically decrease the training size (e.g. set test_size to 0.95)?*

- *Instead of using the datamodel features, you use the PCAs that summarize the influencers, how many top PCs are sufficient to solve this task?*


In [ ]:
ds_test.class_to_idx


{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [ ]:
# example: try 100, 1000, 3000, and 'All'
nfeatures = 0
label = 0
test_size = 0.2 # also try 0.9
seed = 42

In [ ]:
# Load the entire CIFAR10 dataset
cifar_data = np.array(ds_test.data)
target = np.array(ds_test.targets)==label ### Try different sublclasses ###
# Reshape the data to be a 2D array where each row is a sample and each column is a pixel
cifar_data_reshaped = cifar_data.reshape(cifar_data.shape[0], -1)

if nfeatures=='All':
  print('using all features')
  features = cifar_data_reshaped
else:
  print(f'using {nfeatures} features')
  # Pick the same random pixels for each sample
  np.random.seed(seed)
  random_pixel_indices = np.random.choice(cifar_data.shape[1] * cifar_data.shape[2] * cifar_data.shape[3], nfeatures, replace=False) ### Try different numbers of pixels (max is 3072) ###

  # Select the same random pixels for each sample
  features = cifar_data_reshaped[:, random_pixel_indices]

# Balance the dataset by undersampling the majority class
rus = RandomUnderSampler(random_state=seed)
features_resampled, target_resampled = rus.fit_resample(features, target)

# Split the balanced dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, target_resampled, test_size=test_size, random_state=seed) ### Try out different train sizes ###

# Initialize and train the linear classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Compute the validation accuracy
validation_accuracy = accuracy_score(y_test, y_pred)
print(f'Validation Accuracy: {validation_accuracy:.4f}')

In [ ]:
target = np.array(ds_test.targets)==label

if nfeatures=='All':
  print('using all features')
  features = weights.T
else:
  print(f'using {nfeatures} features')
  features = weights.T[:,:nfeatures]

# Balance the dataset by undersampling the majority class
rus = RandomUnderSampler(random_state=seed)
features_resampled, target_resampled = rus.fit_resample(features, target)

# Split the balanced dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, target_resampled, test_size=test_size, random_state=seed)

# Initialize and train the linear classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Compute the validation accuracy
validation_accuracy = accuracy_score(y_test, y_pred)
print(f'Validation Accuracy: {validation_accuracy:.4f}')


### **This example illustrates that we can now learn the simplest model possible - a linear classifier - by using the datamodel embeddings instead of the pixel values and obtain very good and generalizable performance on a task that we never explicitly had labels for when we trained our datamodels.**